# Object Detection using custom trained model

Now that you have trained a custom object detection model, it is time to test it out! 

The codes below are similar to the previous exercise you have done using the pretrained model on COCO dataset. 
Mofify the codes below to use your 'raccoon detector'.

## 1. Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllibcategory_index
import sys
import tarfile
import tensorflow as tf
import zipfile
from shutil import copy2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


## 2. Environment setup

In [ ]:
# root Tensorflow model directory. Modify this accordingly
TF_MODELS_RESEARCH_DIR = '/home/ubuntu/git/tensorflow/models/research'
TF_SLIM_DIR = os.path.join(TF_MODELS_RESEARCH_DIR, 'slim')
TF_OD_DIR = os.path.join(TF_MODELS_RESEARCH_DIR, 'object_detection')

sys.path.append(TF_MODELS_RESEARCH_DIR)
sys.path.append(TF_SLIM_DIR)
sys.path.append(TF_OD_DIR)

## 3. Import TFOD modules


In [ ]:
from utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

## 4. Object Detection on Image

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def run_inference_for_single_image(image_path, graph):
    image = Image.open(image_path)
    
    with graph.as_default():
        with tf.Session() as sess:
        # Get handles to input and output tensors
            image_tensor = graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = graph.get_tensor_by_name('detection_scores:0')
            detection_classes = graph.get_tensor_by_name('detection_classes:0')
            num_detections = graph.get_tensor_by_name('num_detections:0')

            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            [detection_boxes, detection_scores, detection_classes, num_detections],
            # Run inference
            
            (boxes, scores, classes, num) = sess.run(
                            [detection_boxes, detection_scores, detection_classes, num_detections],
                            feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        min_score_thresh=0.4,
                        use_normalized_coordinates=True,
                        line_thickness=10)
            

            # Size, in inches, of the output images.
            IMAGE_SIZE = (12, 8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)

## 4. Model preparation 

Modify the path below to reference your frozen graph, the label map file of the raccoon model. 
Also download a raccoon image from the web and change the image path below to point to your downloaded image (or use the default picture we provided here. 
Run your object detection graph and see how good it can detect the furry animal!

In [ ]:
PATH_TO_FROZEN_GRAPH = '/home/ubuntu/raccoon_project/models/exported_graphs/frozen_inference_graph.pb'
PATH_TO_LABELS = '/home/ubuntu/raccoon_project/data/raccoon_label_map.pbtxt'
IMAGE_PATH = 'sample_image.jpg'  ## download a racoon image from the web and point the path to the file

### Loading label map

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
run_inference_for_single_image(IMAGE_PATH, detection_graph)